# Simple coupled linear regression
*Dario Garcia*

## Problem setting
We have two regression problems on the same input space $\mathbb{R}^k$ which we believe that are very similar. Under linear regression, this assumption translates into similar weight vectors $w_0$ and $w_1$ (i.e. small euclidean distance). How can we exploit this information?

## A simple solution
We can think of learning simultaneously both models while enforcing similarity between them by using an extra additive regularization term that can be interpreted as a simple case of intrinsic Laplacian regularization on an augmented weight space $\mathcal{W}^* = \mathcal{W} \times \mathcal{W}$ which is the cartesian product of the original weight space with itself. The objective function to minimize is then
$$
J(\mathbf{w}) = \sum_{i=1}^N l(w^t\mathbf{x}_i, y_i) + \lambda ||\mathbf{w}||^2 + \lambda_I \mathbf{w}^t L \mathbf{w}
$$
(note that if we let $\lambda_I=0$ we are virtually learning both models independently)

In our case the affinity matrix $A$ for the weights Laplacian matrix has a very simple structure, since we are only coupling analogue weights for the two vectors, i.e. $w_i^0$ and $w_i^1$, so that $A_{ij} = 1$ if $i==j$ or $i+k=j$ or $i-k=j$, $0$ otherwise. Then we can get the Laplacian matrix as $L = D - A$ where $D$ is the degree matrix. We could also absorve the standard regularization term by adding $\lambda \mathbf{I}$ to $L$.

## Numerical optimization
This a nice problem for doing Stochastic Gradient Descent. By doing some basic matrix algebra (which I have probably done wrong ...) we end up with a simple update rule
$$
w_{n+1} \leftarrow w_n - \sigma e_n x_n + \lambda w_n + \lambda_I (w_n - w^c_n) (1-2c_n)
$$
where $e_n$ represents the prediction error at step $n$ and by $w^c$ I mean the "complementary" weight vector (i.e. if we are processing a sample from task 0, $w = w_0$ and $w^c=w_1$ and the other way around) and $c_n$ is the model which is active for sample $n$.

## So let's try this stuff in Python

In [1]:
import numpy as np

K = 20 # Number of dimensions
N = 1000 # Number of points
s = 0.1 # Std of the 'noise' between weights
n_s = 4 # Std of the label noise

# Define the coupled weight vectors
w_0 = np.random.randn(K)
w_1 = w_0 + np.random.randn(K)*s

c = np.random.randint(0,2,100)

# Some input data
X = np.random.randn(K,N)
c = np.concatenate((np.zeros(N/2), np.ones(N/2)))
y_0 = X[:, :N/2].T.dot(w_0) + np.random.randn(N/2) * n_s
y_1 = X[:, N/2:].T.dot(w_1) + np.random.randn(N/2) * n_s
y =  np.concatenate((y_0, y_1))


In [2]:
# The gradient for standard LS linear regression
def basic_gradient(x, w, y, reg):
    pred = x.dot(w)
    err = pred - y
    grad = err * x + reg * w
    return (grad, err)

# Learning parameters
reg = 0.05
learning_rate = 0.0005
num_iter = 100
coupling_reg = 0.1

# Let's estimate independently w_0 and w_1
# BTW, this is the most computationally inefficient way of doing so
print 'Estimating w_0'
w = np.zeros(K)
for iter in range(num_iter):
    cum_err = 0
    for i in range(len(y)/2):
        grad, err = basic_gradient(X[:,i], w, y[i], reg)
        w += -learning_rate * grad
        cum_err += err**2
    if (iter % 10) == 0:
        print 'After iteration %i: %f' % (iter, cum_err)
print 'Error norm (w_1): %.3f' % np.linalg.norm(w-w_0)

print 'Estimating w_1'
w = np.zeros(K)
for iter in range(num_iter):
    cum_err = 0
    for i in range(len(y)/2,len(y)):
        grad, err = basic_gradient(X[:,i], w, y[i], reg)
        w += -learning_rate * grad
        cum_err += err**2
    if (iter % 10) == 0:
        print 'After iteration %i: %f' % (iter, cum_err)
print 'Error norm (w_1): %.3f' % np.linalg.norm(w-w_1)

Estimating w_0
After iteration 0: 18857.137613
After iteration 10: 7984.373495
After iteration 20: 7873.438421
After iteration 30: 7867.739884
After iteration 40: 7867.223297
After iteration 50: 7867.165196
After iteration 60: 7867.157866
After iteration 70: 7867.156873
After iteration 80: 7867.156732
After iteration 90: 7867.156711
Error norm (w_1): 0.690
Estimating w_1
After iteration 0: 18384.608503
After iteration 10: 7567.326397
After iteration 20: 7464.710901
After iteration 30: 7459.598971
After iteration 40: 7459.132193
After iteration 50: 7459.079119
After iteration 60: 7459.072320
After iteration 70: 7459.071379
After iteration 80: 7459.071242
After iteration 90: 7459.071221
Error norm (w_1): 0.707


In [3]:
# The gradient for our coupled problem, in a very explicit form
def coupled_gradient(x, c, w, y, reg, coupling_reg):
    # Note that we are not implementing exactly the formulation presented above,
    # as the L2 norms of the individual weight vectors are decoupled, which makes
    # for sparser and more sensible gradient updates
    k = len(w)
    if c==0:
        w_seg = w[:k/2]
    else:
        w_seg = w[k/2:]
        
    pred = x.dot(w_seg)
    
    coupling = (w[:k/2] - w[k/2:]) * (1 - 2*c)
    err = pred - y
    
    error_grad = np.zeros(k)
    reg_term = np.zeros(k)
    coupling_term = np.zeros(k)
    
    if c==0:
        error_grad[:k/2] = err*x
        reg_term[:k/2] = w_seg
        coupling_term[:k/2] = coupling
    else:
        error_grad[k/2:] = err*x
        reg_term[k/2:] = w_seg
        coupling_term[k/2:] = coupling
        
    grad = error_grad + reg * reg_term + coupling_reg * coupling_term
    
    return (grad, err)

# Now let's learn our shiny coupled model
w = np.zeros(2*K)
for iter in range(num_iter):
    cum_err = 0
    idx = np.random.permutation(len(y))
    for i in idx:
        try:
            grad, err = coupled_gradient(X[:,i], c[i], w, y[i], reg, coupling_reg)
        except:
            print i
            print X[:,i]
            print c[i]
            
        w += -learning_rate * grad
        cum_err += err**2

    if (iter % 10) == 0:
        print 'After iteration %i: %f' % (iter, cum_err)

print 'Error norm (w_0): %.3f' % np.linalg.norm(w[:K]-w_0)
print 'Error norm (w_1): %.3f' % np.linalg.norm(w[K:]-w_1)

print w
print w_0
print w_1

After iteration 0: 37216.741788
After iteration 10: 15555.272857
After iteration 20: 15349.016029
After iteration 30: 15331.675003
After iteration 40: 15328.353843
After iteration 50: 15328.918811
After iteration 60: 15319.686766
After iteration 70: 15331.928695
After iteration 80: 15332.870177
After iteration 90: 15323.133325
Error norm (w_0): 0.646
Error norm (w_1): 0.670
[-1.91906663 -1.51560845 -1.75426357 -1.20230718  0.70756204 -1.63598941
 -0.29701526  0.0248109  -1.27385293 -0.12348197  0.16669     0.07104064
 -0.80939521 -1.43985274  0.96473721  0.08348464  0.66842463 -0.4627149
  0.64865604 -1.91297988 -1.84950529 -1.42873853 -1.65912575 -0.92191313
  0.72207666 -1.49917374 -0.3466662   0.21897691 -1.21587803 -0.01291521
  0.13998422 -0.02644362 -0.56341966 -1.40674759  1.34143764  0.05871314
  0.75409599 -0.47492039  0.74251632 -2.18754426]
[-1.93760735 -1.61733652 -1.63250946 -1.11788623  0.50904929 -1.74010546
 -0.23607633 -0.00250753 -1.22132978 -0.0606537  -0.23752683 -0